# Optimizando el Flujo de Trabajo en Desarrollo y DevOps con Makim

![](https://raw.githubusercontent.com/osl-incubator/makim/main/artwork/v1/logo.png)

## Repositorio

https://github.com/osl-incubator/makim

## Setup

In [ ]:
# !pip install "makim==1.9.1"

## Makim

Problemas que intenta solucionar makim:

* integración con archivos de entorno
* usa un formato amigable (yaml) para la declaración de los targets (scripts) y sus dependencias.
* permite uso de argumentos
* documentación (help) como parte de la estructura
* permite integración con aplicaciones shells y otros lenguajes de programación
* agrupación por grupos y integración de targets de distintos grupos

In [ ]:
%%writefile Makefile

ARG1:=
ARG2:=

target1:
        @echo "target1 $(ARG1)"

target2: target1
        @echo "target2 $(ARG2)"

In [ ]:
!make target1

In [ ]:
! sed -i 's/        /\t/g' Makefile

In [ ]:
!make target1

In [ ]:
!make target2

In [ ]:
# oops no es ARGS1 es ARG1
!make target1 ARGS1=arg1

In [ ]:
!make target1 ARG1=arg1

In [ ]:
# oops no es ARG1 es ARG2
!make target2 ARG1=arg1

In [ ]:
# let's check check the help
!make --help

No hay ninguna información de nuestros targets :(

### Let's play with Makim!

In [ ]:
%%writefile .makim.yaml
version: 1
groups:
  main:
    help: This is the main group, let's have fun
    targets:
      favorite-food:
        help: What is your favorite food?
        args:
          food:
            help: the name of the food
            type: string
            required: true
        run: |
          echo "my favorite food is {{ args.food }}"
      favorite-dessert:
        help: What is your favorite dessert?
        dependencies:
          - target: main.favorite-food
            args:
              food: pizza
          - target: main.favorite-food
            args:
              food: lasagna
        args:
          dessert:
            help: What is your favorite dessert?
            type: string
            required: false
            default: tiramisu
        run: |
          echo "my favorite dessert is {{ args.dessert }}"

Problemas con el format yaml? 
Verifique tu archivo con https://www.yamllint.com/

**Cuáles son los targets disponibles?**

In [ ]:
!makim --help

In [ ]:
!makim main.favorite-food

In [ ]:
!makim main.favorite-food --food quesumacha

In [ ]:
!makim main.favorite-dessert

In [ ]:
!makim main.favorite-dessert --dessert "ice cream"

### Features

Let's have fun!

In [ ]:
%%writefile .env
ENV=dev
SERVICES=s1,s2,s3

In [ ]:
%%writefile docker-compose.yaml
version: "3.9"
services:
  s1:
    image: python:3.11
    command: "python -m http.server 18000"
  s2:
    image: python:3.11
    command: "python -m http.server 18001"
  s3:
    image: python:3.11
    command: "python -m http.server 18002"

NOTA: necesitamos instalar docker-compose. Para ello, instalamos el paquete compose-go que es un wrapper para docker-compose 2 escrito go.

In [ ]:
# !pip install compose-go

In [ ]:
%%writefile .makim.yaml
version: 1

groups:
  docker:
    help: control the docker from makim
    targets:
      up:
        help: Start the services
        run: |
          docker compose up -d
      stop:
        help: Stop the services
        run: |
          docker compose stop

      config-with-bash:
        help: show information for the services using bash and using jinja2
        env-file:
          .env
        shell: bash
        run: |
          {% for service in env.SERVICES.split(",") %}
          docker compose config "{{service}}"
          {% endfor %}

      config-with-xonsh:
        help: show information for the services using xonsh (https://xon.sh/bash_to_xsh.html?highlight=env)
        env-file:
          .env
        run: |
          for service in {{ env.SERVICES.split(",") }}:
            docker compose config @(service)

  path:
    help: targets that handles path
    targets:
      workdir:
        help: shows how to use the working directory
        working-directory: /tmp
        run:
          pwd

  python:
    help: "some commands using python as interpreter"
    shell: python
    targets:
      test-a-gt-b:
        help: target that shows how to use assert
        args:
          a:
            help: Argument a
            type: int
            required: true
          b:
            help: Argument b
            type: int
            required: true
        run: assert {{ args.a }} > {{ args.b }}
      debug:
        help: target that shows how to use assert
        run: |
          breakpoint()
          print("breakpoint works!")
              

In [ ]:
!makim --help

In [ ]:
!makim docker.up

In [ ]:
!makim docker.config-with-bash

In [ ]:
!makim docker.config-with-xonsh

In [ ]:
!makim path.workdir

In [ ]:
!makim python.test-a-gt-b

In [ ]:
!makim python.test-a-gt-b --a 2 --b 3

In [ ]:
!makim python.test-a-gt-b --a 2 --b 1

In [ ]:
!pwd

In [ ]:
# %%debug
# makim python.debug